# Utiliser des LLM en API pour générer avec une prompt (zero shot)

- des annotations : quels sont les articles qui mentionnent des thématiques IA
- de l'extraction d'information : quelles sont les notions utilisées

Quelques lectures 

- Stuhler, O., Ton, C. D., & Ollion, E. (2025). From Codebooks to Promptbooks: Extracting Information from Text with Generative Large Language Models. Sociological Methods & Research, 54(3), 794-848. https://doi.org/10.1177/00491241251336794 (Original work published 2025)
- https://www.css.cnrs.fr/classification-with-generative-llms-and-api-calls/

## Quelques mots sur le génératif

- Prédiction du next token
- Modèles pré-entrainés
    - des tailles & alignements différents
- Dépendances matérielles fortes

## Quels modèles ?

- vaste question
- équilibre puissance/coût (financier et matériel)
- enjeu de la sécurité des données

De nombreux prestataires

- openai, etc.
- openrouter
- humanum...

## Charger les données

In [1]:
import pandas as pd
df = pd.read_excel("../data/data_hal_crest_clean.xlsx")
df.head()

,Unnamed: 0,docid,title,abstract,abstract_eng,publicationDate_tdate,document_type,authors,date,abstract_net,title_net,texte,taille,cont_IA,cont_algo,cont_ml
0,0,3026577,"[""The effect of price magnitude on analysts' f...",['Recent research in finance shows that the ma...,['Recent research in finance shows that the ma...,2021-01-01T00:00:00Z,shs.eco,"['Tristan Roger', 'Wael Bousselmi', 'Patrick R...",2021-01-01,Recent research in finance shows that the magn...,The effect of price magnitude on analysts' for...,The effect of price magnitude on analysts' for...,822,False,False,False
1,1,4358212,['Model-based Clustering with Missing Not At R...,"['Model-based unsupervised learning, as any le...","['Model-based unsupervised learning, as any le...",2024-06-18T00:00:00Z,stat.ml,"['Aude Sportisse', 'Matthieu Marbac', 'Fabien ...",2024-06-18,"Model-based unsupervised learning, as any lear...",Model-based Clustering with Missing Not At Ran...,Model-based Clustering with Missing Not At Ran...,1409,False,True,False
2,2,4996040,['Inferring the parameters of Taylor’s power l...,['Taylor’s power law of fluctuation scaling ha...,['Taylor’s power law of fluctuation scaling ha...,2025-11-26T00:00:00Z,stat,"['Lionel Truquet', 'Joel E. Cohen', 'Paul Douk...",2025-11-26,Taylor’s power law of fluctuation scaling has ...,Inferring the parameters of Taylor’s power law...,Inferring the parameters of Taylor’s power law...,1570,False,False,False
3,3,5357551,['Nonparametric intensity estimation of spatia...,['We propose a random forest estimator for the...,['We propose a random forest estimator for the...,2025-11-10T00:00:00Z,stat.me,"['Christophe Biscio', 'Frédéric Lavancier']",2025-11-10,We propose a random forest estimator for the i...,Nonparametric intensity estimation of spatial ...,Nonparametric intensity estimation of spatial ...,814,False,False,False
4,4,3026615,['Impact of the Brexit vote announcement on lo...,['We examine how the Brexit announcement influ...,['We examine how the Brexit announcement influ...,2020-01-01T00:00:00Z,shs.eco,"['Wael Bousselmi', 'Patrick Sentis', 'Marc Wil...",2020-01-01,We examine how the Brexit announcement influen...,Impact of the Brexit vote announcement on long...,Impact of the Brexit vote announcement on long...,759,False,False,False


## Charger une clé

In [2]:
import yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
api_key = config["api_key"]

## Utiliser openrouter et l'API openapi

Quickstart : https://platform.openai.com/docs/quickstart?api-mode=chat&language=python

In [3]:
#!pip install openai
from openai import OpenAI
api_url = "https://openrouter.ai/api/v1"
 
client = OpenAI(
  base_url=api_url,
  api_key=api_key,
)

response = client.chat.completions.create(
    model="meta-llama/llama-3.3-70b-instruct",
    messages=[{"role": "user", "content": "Hello, Llama. Say Hi!"}]
)
response

ChatCompletion(id='gen-1770048186-aGVQuh1xRP0i75TZEps0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1770048196, model='meta-llama/llama-3.3-70b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=25, prompt_tokens=19, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0), cost=1.668e-05, is_byok=False, cost_details={'upstream_inference_cost': 1.668e-05, 'upstream_inference_prompt_cost': 4.18e-06, 'upstream_inference_completio

Choisir un modèle

Construire la prompt

In [ ]:
messages = [
  {
    "role": "system",
    "content": "You are an efficient research assistant helping with text annotation."
  },
  {
    "role": "user",
    "content": """
    Annotate this following text if its topic is about AI or algorithms.
    If so, returns AI, and otherwise, return NOT AI.
    """
      + df["abstract"].iloc[1]
  }
]

In [30]:
#!pip install openai
from openai import OpenAI
api_url = "https://openrouter.ai/api/v1"
 
client = OpenAI(
  base_url=api_url,
  api_key=api_key,
)

response = client.chat.completions.create(
    model="meta-llama/llama-3.3-70b-instruct",
    messages=messages
)
response.choices[0].message.content

'The topic of the text is about AI.\n\nThe returned value is: **AI**\n\nThe keywords associated with AI in the text are:\n\n* Machine learning (specifically, "unsupervised learning")\n* Model-based clustering algorithms\n* Mixture model\n* Expectation Maximization algorithm\n* Statistical inference\n* Data distribution\n* MNAR (Missing Not At Random) models\n\nThese keywords indicate that the text is related to artificial intelligence and machine learning, particularly in the context of handling missing data and developing algorithms for clustering and statistical analysis.'

### Application Faire une boucle sur une liste de documents : évaluer si un abstract porte ou pas sur l'IA

- Tester avec des exemples (few shot)
- Tester avec du raisonnement (chaîne of thought)

## Evaluation

Comment évaluer la qualité d'un codage ? 

- La nécessité d'un gold standard
- Le calcul de métriques `from sklearn.metrics import classification_report`

### Application : construire un jeu de données d'évaluation

- Utiliser des outils permettant l'interaction humaine
- Toujours de l'ambiguité : accord inter-annotateur (alpha de Chronbach, autre)

### Mesurer la qualité d'une évaluation

Différentes métriques

- précision
- recall
- f1

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Application : extraire de l'information

Sur les articles identifiés comme AI-related, extraire les termes liés pour les analyser

- Construire une prompt
- Tester sur plusieurs textes
- Gérer la structure des données